## IMPUTE INSANITY!:
### before we move onto neural networks we need to impute the NANs
#### How do we even know we've correctly imputed?
#### **Try out every impute methods and per feature set train the GBM and see which one got most importance**

### TODO
- #### [KNN , DeepLearner, other novel ways to impute](https://towardsdatascience.com/6-different-ways-to-compensate-for-missing-values-data-imputation-with-examples-6022d9ca0779)
  - KNN thoughts: we can perhaps use subset of features with no nans, but then the records will become similar, and decision trees already predict the same result for similar looking rows? then what's the point of imputing value same as similar looking rows?
- ### DON't Impute rather provide the Flag wether the feature has value or not, kinda like mask; we did this in Optiver (not the transformer sequence mask , but we have mask about weather the second/grouped seconds is missing or not)
  - #### [Results on passing information about missing values to neural networks](https://machinelearningmastery.com/binary-flags-for-missing-values-for-machine-learning/)
- ### AMEND ABOVE: Impute as well as pass the missing Flag too, so neural network learns better
- ### Categorical Imputation: train on data with missing categorical values as target variable! **We can even use TEST set to do this!!!! yay!**
  - #### [Training to find missing categorical values](https://www.analyticsvidhya.com/blog/2021/04/how-to-handle-missing-values-of-categorical-variables/)

### IMPORTANT FLOAT16 aggregations return NONE!!! mean, sum most of it doesn't work :-/

## TODO:
- ### MULTILEVEL IMPUTATION DEFAULT
- ### LEVEL 1: out of all the variant+original; select the one with best amex scores DEFAULT_EMP1
    - #### This is partial nans
- ### Level 2: now that we have partial none df with best variants(including original with nans) ; TRAIN ML MODEL to have PREDICTED/BS variant
    - #### In the order of lowest missing values to highest missing values
    - #### we Target the feature with lowest amount on NAN as we have maximum training rows; then slowly cascade to more missing row features
    - #### once we have computed PREDICTED/BS variants for all of the features with nans
- ### Level 3: from DEFAULT_EMP1 vs PREDICTED variant select the one with maximum amex score (this is kinda like back to stage 1 but with improved train_df_wt)
    - #### repetetively select and choose the best varint out of the two
    - #### DEFAULT_EMP2 is generated!
    - #### Now there is a scope of repeating Level 2 again with using base_df of EMP2 and then going to EMP3,

In [1]:
WHERE_THIS = "local" # local|kaggle

BS_DATASET = "amex-default-prediction-binarysentient"
if WHERE_THIS == "kaggle":
    INPUT_PATH = "/kaggle/input/amex-default-prediction/"
    OUTPUT_PATH = "/kaggle/working/"
    TEMP_PATH = "/kaggle/temp/"
elif WHERE_THIS == "local":
    INPUT_PATH = "input/amex-default-prediction/"
    OUTPUT_PATH = "working/"
    TEMP_PATH = "temp/"


In [166]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import gc
import os
from datetime import datetime
import torch


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from IPython.display import display, HTML
from lightgbm import LGBMClassifier, log_evaluation
from sklearn.model_selection import StratifiedKFold
import plotly.express as pltex

import impute_insanity
from impute_insanity import load_prepare_amex_dataset
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


for dirname, _, filenames in os.walk(INPUT_PATH):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

input/amex-default-prediction/sample_submission.csv
input/amex-default-prediction/test_data.csv
input/amex-default-prediction/train_data.csv
input/amex-default-prediction/train_labels.csv
input/amex-default-prediction/.ipynb_checkpoints\sample_submission-checkpoint.csv


In [132]:
object_features = ['customer_ID']
datetime_features = ['S_2']
target_features = ['target']
cat_features = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
numeric_features = ['R_2', 'S_25', 'D_118', 'B_5', 'D_60', 'B_21', 'D_115', 'S_15', 'D_84', 'D_122', 'B_9', 'B_33', 'D_53', 'R_24', 'D_94', 'D_56', 'D_139', 'R_28', 'B_3', 'S_20', 'B_31', 'D_133', 'B_2', 'S_12', 'D_71', 'D_96', 'S_7', 'D_72', 'B_36', 'B_41', 'S_5', 'D_41', 'R_22', 'R_8', 'D_140', 'D_47', 'D_89', 'P_2', 'R_19', 'D_59', 'B_23', 'S_3', 'D_145', 'D_103', 'B_19', 'R_20', 'D_73', 'D_136', 'D_141', 'D_142', 'B_22', 'D_46', 'B_29', 'B_25', 'D_128', 'B_18', 'D_86', 'D_109', 'B_8', 'B_17', 'R_17', 'B_12', 'D_54', 'D_74', 'S_16', 'B_6', 'D_49', 'D_80', 'S_8', 'B_7', 'D_144', 'B_27', 'B_26', 'R_25', 'R_23', 'D_82', 'D_111', 'B_10', 'D_113', 'R_4', 'D_48', 'R_26', 'S_23', 'B_11', 'D_104', 'D_134', 'D_79', 'P_3', 'D_132', 'D_137', 'D_135', 'B_28', 'D_88', 'S_13', 'D_51', 'D_61', 'D_75', 'D_69', 'R_16', 'S_6', 'S_17', 'D_93', 'B_20', 'D_112', 'D_123', 'D_130', 'B_1', 'D_78', 'D_92', 'S_27', 'D_44', 'B_16', 'R_5', 'D_43', 'S_18', 'B_15', 'D_39', 'D_50', 'D_55', 'S_9', 'D_105', 'D_70', 'R_18', 'D_125', 'D_58', 'S_24', 'D_110', 'D_42', 'R_6', 'D_81', 'R_7', 'D_138', 'D_52', 'R_27', 'D_124', 'D_45', 'D_91', 'D_108', 'S_22', 'B_14', 'D_83', 'R_13', 'D_87', 'S_19', 'D_131', 'R_21', 'B_40', 'R_3', 'D_65', 'B_13', 'D_129', 'D_119', 'B_32', 'R_9', 'B_24', 'D_127', 'D_106', 'D_102', 'R_1', 'R_14', 'B_37', 'D_107', 'R_15', 'R_12', 'P_4', 'R_10', 'customer_ID', 'D_121', 'R_11', 'S_11', 'D_76', 'D_143', 'B_39', 'B_42', 'D_62', 'B_4', 'S_26', 'D_77']
    

In [5]:
def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

## TODO: FAST AMEX implementation is not accurate; convert the dataframe accurate version to directly work with numpy arrays
# @yunchonggan's fast metric implementation
# From https://www.kaggle.com/competitions/amex-default-prediction/discussion/328020
def fast_amex_metric(y_true: np.array, y_pred: np.array) -> float:

    # count of positives and negatives
    n_pos = y_true.sum()
    n_neg = y_true.shape[0] - n_pos

    # sorting by descring prediction values
    indices = np.argsort(y_pred)[::-1]
    preds, target = y_pred[indices], y_true[indices]

    # filter the top 4% by cumulative row weights
    weight = 20.0 - target * 19.0
    cum_norm_weight = (weight / weight.sum()).cumsum()
    four_pct_filter = cum_norm_weight <= 0.04

    # default rate captured at 4%
    d = target[four_pct_filter].sum() / n_pos

    # weighted gini coefficient
    lorentz = (target / n_pos).cumsum()
    gini = ((lorentz - cum_norm_weight) * weight).sum()

    # max weighted gini coefficient
    gini_max = 10 * n_neg * (1 - 19 / (n_pos + 20 * n_neg))

    # normalized weighted gini coefficient
    g = gini / gini_max

    return 0.5 * (g + d)

# Need Lightgbm supported eval metric
#Custom eval function expects a callable with following signatures: func(y_true, y_pred), func(y_true, y_pred, weight) or func(y_true, y_pred, weight, group) and returns (eval_name, eval_result, is_higher_better) or list of (eval_name, eval_result, is_higher_better):
def lgbm_eval_metric_amex(y_true, y_pred):
    amex_metric = fast_amex_metric(y_true, y_pred)
    return ('amex', amex_metric, True)

In [6]:
train_label_df = load_prepare_amex_dataset('train_labels')#, index_col='customer_ID')
train_label_df

,customer_ID,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0.0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0.0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0.0
...,...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,0.0
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,0.0
458910,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,0.0
458911,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,1.0


In [7]:
prediction_labels = train_label_df.copy()
# prediction_labels['prediction'] = prediction_labels['target'] 
# prediction_labels['prediction'] = prediction_labels['target'] - 3
prediction_labels['prediction'] = 1000
del prediction_labels['target'] 


### Predictions impact on Evaluation Metric
- #### all predictions 0 :   score 0.018829875288988086
- #### all predictions 1 :   score 0.018829875288988086
- #### all predictions 0.4 : score 0.018829875288988086
- #### same as target      : score 1.0
- #### target * 10         : score 1.0
- #### target * 0.001      : score 1.0
- #### target - 3          : score 1.0
- #### target + 3          : score 1.0

## The evaluation metrics is solid! amazing, need to understand more
## TODO: FAST AMEX implementation is not accurate; convert the dataframe accurate version to directly work with numpy arrays

In [8]:
## TODO: FAST AMEX implementation is not accurate; convert the dataframe accurate version to directly work with numpy arrays
# print(amex_metric(train_labels, prediction_labels))
print(fast_amex_metric(train_label_df['target'], prediction_labels['prediction']))

0.02260958442726859


In [9]:
def create_model_with_config(random_state=1, n_estimators=1200, importance_type=None, early_stopping_rounds=None):
    """
    Creates model with our desired and some default hyper params
    importance_type: split|gain
    """
    return LGBMClassifier(n_estimators=n_estimators,
                          learning_rate=0.03, reg_lambda=50,
                          min_child_samples=2400,
                          num_leaves=95,
                          colsample_bytree=0.19,early_stopping_rounds=early_stopping_rounds,
                          max_bins=511, random_state=random_state, importance_type=importance_type)

## Precision impact on scores:

### float32: 0.788541
### float16: 0.788748, 0.788649, 0.788821
### float64: 0.788276, 0.789053, 0.789025, 0.788543


## Default

## EMP1


In [10]:
fold_score_track = {}
def track_score(key, score):
    key = str(key)
    if key not in fold_score_track:
        fold_score_track[key] = []
    fold_score_track[key].append(score)

def show_score(key):
    key = str(key)
    display(HTML(f"<h3>{key} OVERALL SCORE : {np.mean(fold_score_track[key]):0.6f}</h3>"))

CPU times: total: 3min 55s
Wall time: 3min 55s


In [57]:
def create_lgbm_model_with_config(random_state=1, n_estimators=1200, importance_type=None, early_stopping_rounds=None):
    """
    Creates model with our desired and some default hyper params
    importance_type: split|gain
    """
    if early_stopping_rounds is None:
        early_stopping_rounds = n_estimators//5
    return LGBMClassifier(n_estimators=n_estimators,
                          learning_rate=0.03, reg_lambda=50,
                          min_child_samples=2400,
                          num_leaves=95, num_threads=12,
                          colsample_bytree=0.19,early_stopping_rounds=early_stopping_rounds,
                          max_bins=511, random_state=random_state, importance_type=importance_type)


In [179]:
%%time
features_avg = ['B_1', 'B_2', 'B_3', 'B_4', 'B_5', 'B_6', 'B_8', 'B_9', 'B_10', 'B_11', 'B_12', 'B_13', 'B_14', 'B_15', 'B_16', 'B_17', 'B_18', 'B_19', 'B_20', 'B_21', 'B_22', 'B_23', 'B_24', 'B_25', 'B_28', 'B_29', 'B_30', 'B_32', 'B_33', 'B_37', 'B_38', 'B_39', 'B_40', 'B_41', 'B_42', 'D_39', 'D_41', 'D_42', 'D_43', 'D_44', 'D_45', 'D_46', 'D_47', 'D_48', 'D_50', 'D_51', 'D_53', 'D_54', 'D_55', 'D_58', 'D_59', 'D_60', 'D_61', 'D_62', 'D_65', 'D_66', 'D_69', 'D_70', 'D_71', 'D_72', 'D_73', 'D_74', 'D_75', 'D_76', 'D_77', 'D_78', 'D_80', 'D_82', 'D_84', 'D_86', 'D_91', 'D_92', 'D_94', 'D_96', 'D_103', 'D_104', 'D_108', 'D_112', 'D_113', 'D_114', 'D_115', 'D_117', 'D_118', 'D_119', 'D_120', 'D_121', 'D_122', 'D_123', 'D_124', 'D_125', 'D_126', 'D_128', 'D_129', 'D_131', 'D_132', 'D_133', 'D_134', 'D_135', 'D_136', 'D_140', 'D_141', 'D_142', 'D_144', 'D_145', 'P_2', 'P_3', 'P_4', 'R_1', 'R_2', 'R_3', 'R_7', 'R_8', 'R_9', 'R_10', 'R_11', 'R_14', 'R_15', 'R_16', 'R_17', 'R_20', 'R_21', 'R_22', 'R_24', 'R_26', 'R_27', 'S_3', 'S_5', 'S_6', 'S_7', 'S_9', 'S_11', 'S_12', 'S_13', 'S_15', 'S_16', 'S_18', 'S_22', 'S_23', 'S_25', 'S_26']
features_avg = list(set(features_avg).difference(set(cat_features)))
features_min = ['B_2', 'B_4', 'B_5', 'B_9', 'B_13', 'B_14', 'B_15', 'B_16', 'B_17', 'B_19', 'B_20', 'B_28', 'B_29', 'B_33', 'B_36', 'B_42', 'D_39', 'D_41', 'D_42', 'D_45', 'D_46', 'D_48', 'D_50', 'D_51', 'D_53', 'D_55', 'D_56', 'D_58', 'D_59', 'D_60', 'D_62', 'D_70', 'D_71', 'D_74', 'D_75', 'D_78', 'D_83', 'D_102', 'D_112', 'D_113', 'D_115', 'D_118', 'D_119', 'D_121', 'D_122', 'D_128', 'D_132', 'D_140', 'D_141', 'D_144', 'D_145', 'P_2', 'P_3', 'R_1', 'R_27', 'S_3', 'S_5', 'S_7', 'S_9', 'S_11', 'S_12', 'S_23', 'S_25']
features_min = list(set(features_min).difference(set(cat_features)))
features_max = ['B_1', 'B_2', 'B_3', 'B_4', 'B_5', 'B_6', 'B_7', 'B_8', 'B_9', 'B_10', 'B_12', 'B_13', 'B_14', 'B_15', 'B_16', 'B_17', 'B_18', 'B_19', 'B_21', 'B_23', 'B_24', 'B_25', 'B_29', 'B_30', 'B_33', 'B_37', 'B_38', 'B_39', 'B_40', 'B_42', 'D_39', 'D_41', 'D_42', 'D_43', 'D_44', 'D_45', 'D_46', 'D_47', 'D_48', 'D_49', 'D_50', 'D_52', 'D_55', 'D_56', 'D_58', 'D_59', 'D_60', 'D_61', 'D_63', 'D_64', 'D_65', 'D_70', 'D_71', 'D_72', 'D_73', 'D_74', 'D_76', 'D_77', 'D_78', 'D_80', 'D_82', 'D_84', 'D_91', 'D_102', 'D_105', 'D_107', 'D_110', 'D_111', 'D_112', 'D_115', 'D_116', 'D_117', 'D_118', 'D_119', 'D_121', 'D_122', 'D_123', 'D_124', 'D_125', 'D_126', 'D_128', 'D_131', 'D_132', 'D_133', 'D_134', 'D_135', 'D_136', 'D_138', 'D_140', 'D_141', 'D_142', 'D_144', 'D_145', 'P_2', 'P_3', 'P_4', 'R_1', 'R_3', 'R_5', 'R_6', 'R_7', 'R_8', 'R_10', 'R_11', 'R_14', 'R_17', 'R_20', 'R_26', 'R_27', 'S_3', 'S_5', 'S_7', 'S_8', 'S_11', 'S_12', 'S_13', 'S_15', 'S_16', 'S_22', 'S_23', 'S_24', 'S_25', 'S_26', 'S_27']
features_max = list(set(features_max).difference(set(cat_features)))
features_last = ['B_1', 'B_2', 'B_3', 'B_4', 'B_5', 'B_6', 'B_7', 'B_8', 'B_9', 'B_10', 'B_11', 'B_12', 'B_13', 'B_14', 'B_15', 'B_16', 'B_17', 'B_18', 'B_19', 'B_20', 'B_21', 'B_22', 'B_23', 'B_24', 'B_25', 'B_26', 'B_28', 'B_29', 'B_30', 'B_32', 'B_33', 'B_36', 'B_37', 'B_38', 'B_39', 'B_40', 'B_41', 'B_42', 'D_39', 'D_41', 'D_42', 'D_43', 'D_44', 'D_45', 'D_46', 'D_47', 'D_48', 'D_49', 'D_50', 'D_51', 'D_52', 'D_53', 'D_54', 'D_55', 'D_56', 'D_58', 'D_59', 'D_60', 'D_61', 'D_62', 'D_63', 'D_64', 'D_65', 'D_69', 'D_70', 'D_71', 'D_72', 'D_73', 'D_75', 'D_76', 'D_77', 'D_78', 'D_79', 'D_80', 'D_81', 'D_82', 'D_83', 'D_86', 'D_91', 'D_96', 'D_105', 'D_106', 'D_112', 'D_114', 'D_119', 'D_120', 'D_121', 'D_122', 'D_124', 'D_125', 'D_126', 'D_127', 'D_130', 'D_131', 'D_132', 'D_133', 'D_134', 'D_138', 'D_140', 'D_141', 'D_142', 'D_145', 'P_2', 'P_3', 'P_4', 'R_1', 'R_2', 'R_3', 'R_4', 'R_5', 'R_6', 'R_7', 'R_8', 'R_9', 'R_10', 'R_11', 'R_12', 'R_13', 'R_14', 'R_15', 'R_19', 'R_20', 'R_26', 'R_27', 'S_3', 'S_5', 'S_6', 'S_7', 'S_8', 'S_9', 'S_11', 'S_12', 'S_13', 'S_16', 'S_19', 'S_20', 'S_22', 'S_23', 'S_24', 'S_25', 'S_26', 'S_27']
features_last = list(set(features_last).union(set(cat_features)))
for i in ['test', 'train']:
    df = load_prepare_amex_dataset(f"{i}_data_emp1")
    cid = pd.Categorical(df.pop('customer_ID'), ordered=True)
    last = (cid != np.roll(cid, -1)) # mask for last statement of every customer
    if 'target' in df.columns:
        df.drop(columns=['target'], inplace=True)
    gc.collect()
    print('Read', i)
    df_avg = (df
              .groupby(cid)
              .mean()[features_avg]
              .rename(columns={f: f"{f}_avg" for f in features_avg})
             )
    gc.collect()
    print('Computed avg', i)
    df_min = (df
              .groupby(cid)
              .min()[features_min]
              .rename(columns={f: f"{f}_min" for f in features_min})
             )
    gc.collect()
    print('Computed min', i)
    df_max = (df
              .groupby(cid)
              .max()[features_max]
              .rename(columns={f: f"{f}_max" for f in features_max})
             )
    gc.collect()
    print('Computed max', i)
    df = (df.loc[last, features_last]
          .rename(columns={f: f"{f}_last" for f in features_last})
          .set_index(np.asarray(cid[last]))
         )
    gc.collect()
    print('Computed last', i)
    df = pd.concat([df, df_min, df_max, df_avg], axis=1)
    if i == 'train': df_train = df
    else: df_test = df
    print(f"{i} shape: {df.shape}")
    del df, df_avg, df_min, df_max, cid, last

target = load_prepare_amex_dataset('train_labels').target.values
print(f"target shape: {target.shape}")

Read test
Computed avg test


<timed exec>:26: FutureWarning: Dropping invalid columns in DataFrameGroupBy.min is deprecated. In a future version, a TypeError will be raised. Before calling .min, select only columns which should be valid for the function.


Computed min test


<timed exec>:33: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.


Computed max test
Computed last test
test shape: (924621, 459)
Read train
Computed avg train


<timed exec>:26: FutureWarning: Dropping invalid columns in DataFrameGroupBy.min is deprecated. In a future version, a TypeError will be raised. Before calling .min, select only columns which should be valid for the function.


Computed min train


<timed exec>:33: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.


Computed max train
Computed last train
train shape: (458913, 459)
target shape: (458913,)
CPU times: total: 3min 12s
Wall time: 1min 52s


In [87]:
for dataset in ['train_data','train_data_emp1', 'test_data','test_data_emp1']:
    df = load_prepare_amex_dataset(dataset)
    nanmap = df.isna().sum().reset_index().rename({'index':'feature',0:'nan_count'}, axis=1).to_dict('records')
    print(dataset, "nan", len([x for x in nanmap if x['nan_count']>0]))
    del df
gc.collect()

train_data nan 122
train_data_emp1 nan 51
test_data nan 121
test_data_emp1 nan 50


0

In [177]:
for dataset in ['train_data','train_data_emp1', 'test_data','test_data_emp1']:
    df = load_prepare_amex_dataset(dataset)
    nanmap = df.isna().sum().reset_index().rename({'index':'feature',0:'nan_count'}, axis=1).to_dict('records')
    print(dataset, "nan", len([x for x in nanmap if x['nan_count']>0]))
    del df
gc.collect()

train_data nan 122
train_data_emp1 nan 35
test_data nan 121
test_data_emp1 nan 38


0

In [ ]:
%%time
# variant_df = variant_df[reversed(list(variant_df.columns))]
# we keep this copy so that original feature doens't stay on top!!!
# it's some weird quirk i found
# variant_df[missingvaluefeature+"_COPY"] = variant_df[missingvaluefeature]
# del variant_df[missingvaluefeature]
features_x = [f for f in df_train.columns if f != 'customer_ID' and f != 'target' and f!='S_2']
feature_y = 'target'
# df_x = df_train[features_x]
# df_y = df_train_wt[feature_y]
# target =

total_splits = 5
kf = StratifiedKFold(n_splits=total_splits, shuffle=True)
fold_scores = []
models = []
# NOT TOO SURE about feature_importances when all features are variants!! it's non decisive
for fold, (idx_train, idx_dev) in enumerate(kf.split(df_train, target)):
    print(f"------ FOLD: {fold+1} of {total_splits} -------")
    train_x, dev_x, train_y, dev_y, model = None, None, None, None, None
    start_time = datetime.now()

    X_tr = df_train.iloc[idx_train][features_x]
    X_va = df_train.iloc[idx_dev][features_x]
    y_tr = target[idx_train]
    y_va = target[idx_dev]
   
    model = create_lgbm_model_with_config(importance_type='gain')
    model.fit(X_tr, y_tr,
                  eval_set = [(X_va, y_va)], 
                  eval_metric=[lgbm_eval_metric_amex],
                  callbacks=[log_evaluation(100)])

    y_va_pred = model.predict_proba(X_va, raw_score=True)
    score = fast_amex_metric(y_va, y_va_pred)
    n_trees = model.best_iteration_
    if n_trees is None: n_trees = model.n_estimators
    print("-------------------------------------")
    display(HTML(f"<h3>Fold {fold+1} | {str(datetime.now() - start_time)[-12:-7]} |"
          f" {n_trees:5} trees |"
          f"                Score = {score:.5f} | Importance: Gain</h3>"))
    print("-------------------------------------")
    # fold_scores.append(score)
    track_score(features_x, score)
    models.append(model)
    # feature_importance_tuples = sorted(zip(X_tr.columns,model.feature_importances_), key=lambda x:x[1])
    # # feature_importance_tuples = sorted(zip(X_tr.columns,model.feature_importance(importance_type='gain')), key=lambda x:x[1])
    # feature_importance_map = {k:v for k,v in feature_importance_tuples}
    # feature_imp = pd.DataFrame(feature_importance_tuples, columns=['feature','importance'])
    # fig = pltex.bar(feature_imp, x='importance', y='feature', height=500)
    # fig.show()


    # track_score(features_x, score)
            # feature_importance_tuples = sorted(zip(X_tr.columns,model.feature_importances_), key=lambda x:x[1])
            # # feature_importance_tuples = sorted(zip(X_tr.columns,model.feature_importance(importance_type='gain')), key=lambda x:x[1])
            # feature_importance_map = {k:v for k,v in feature_importance_tuples}
            # feature_imp = pd.DataFrame(feature_importance_tuples, columns=['feature','importance'])
            # fig = pltex.bar(feature_imp, x='importance', y='feature', height=500)
            # fig.show()

show_score(features_x)
        
    # input("-------- ENTER TO CONINUE -------")
       

    

------ FOLD: 1 of 5 -------
[LightGBM] [Warning] num_threads is set=12, n_jobs=-1 will be ignored. Current value: num_threads=12
[LightGBM] [Warning] early_stopping_round is set=240, early_stopping_rounds=240 will be ignored. Current value: early_stopping_round=240


C:\Users\nisha\miniconda3\envs\kaggle\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\nisha\miniconda3\envs\kaggle\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[100]	valid_0's binary_logloss: 0.243287	valid_0's amex: 0.771298
[200]	valid_0's binary_logloss: 0.223107	valid_0's amex: 0.783541
[300]	valid_0's binary_logloss: 0.218108	valid_0's amex: 0.789187
[400]	valid_0's binary_logloss: 0.215963	valid_0's amex: 0.791742
[500]	valid_0's binary_logloss: 0.214869	valid_0's amex: 0.793569
[600]	valid_0's binary_logloss: 0.21419	valid_0's amex: 0.793555
[700]	valid_0's binary_logloss: 0.2138	valid_0's amex: 0.79423
[800]	valid_0's binary_logloss: 0.213579	valid_0's amex: 0.79508
[900]	valid_0's binary_logloss: 0.213386	valid_0's amex: 0.795875
[1000]	valid_0's binary_logloss: 0.213252	valid_0's amex: 0.79714
[1100]	valid_0's binary_logloss: 0.213187	valid_0's amex: 0.796124
[1200]	valid_0's binary_logloss: 0.21313	valid_0's amex: 0.796311
-------------------------------------


-------------------------------------
------ FOLD: 2 of 5 -------
[LightGBM] [Warning] num_threads is set=12, n_jobs=-1 will be ignored. Current value: num_threads=12
[LightGBM] [Warning] early_stopping_round is set=240, early_stopping_rounds=240 will be ignored. Current value: early_stopping_round=240


C:\Users\nisha\miniconda3\envs\kaggle\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\nisha\miniconda3\envs\kaggle\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


...

C:\Users\nisha\miniconda3\envs\kaggle\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\nisha\miniconda3\envs\kaggle\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[100]	valid_0's binary_logloss: 0.246879	valid_0's amex: 0.76426
[200]	valid_0's binary_logloss: 0.22754	valid_0's amex: 0.780273
[300]	valid_0's binary_logloss: 0.222654	valid_0's amex: 0.786762
[400]	valid_0's binary_logloss: 0.220674	valid_0's amex: 0.789562
[500]	valid_0's binary_logloss: 0.219611	valid_0's amex: 0.79004
[600]	valid_0's binary_logloss: 0.218983	valid_0's amex: 0.790909
[700]	valid_0's binary_logloss: 0.2186	valid_0's amex: 0.791825
[800]	valid_0's binary_logloss: 0.218321	valid_0's amex: 0.790938
[900]	valid_0's binary_logloss: 0.218186	valid_0's amex: 0.79141
-------------------------------------


-------------------------------------
------ FOLD: 5 of 5 -------
[LightGBM] [Warning] num_threads is set=12, n_jobs=-1 will be ignored. Current value: num_threads=12
[LightGBM] [Warning] early_stopping_round is set=240, early_stopping_rounds=240 will be ignored. Current value: early_stopping_round=240


C:\Users\nisha\miniconda3\envs\kaggle\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\nisha\miniconda3\envs\kaggle\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[100]	valid_0's binary_logloss: 0.246518	valid_0's amex: 0.764719
[200]	valid_0's binary_logloss: 0.227155	valid_0's amex: 0.780172


In [85]:
gc.collect()

18763

In [23]:
%%time
# df_test = load_prepare_amex_dataset("test_data_emp1")
# df_test_last1 = df_test.groupby('customer_ID').last()
# df_test_last1 = df_test_last1.reset_index()
# df_test_wt = pd.merge(df_test_last1, train_label_df, how='inner', on = 'customer_ID')#.reset_index()

CPU times: total: 9min 3s
Wall time: 8min 2s


In [59]:
features_x = [f for f in df_test.columns if f not in ['customer_ID', 'target', 'prediction', 'S_2']]
X_tr = df_test[features_x]

predictions = []
for model in models:
    preds = model.predict_proba(X_tr)
    predictions.append(preds[:,1])#/len(models)
    


924621

In [66]:
df_test_last1['prediction'] = np.max(predictions, axis=0)
df_test_last1[['customer_ID','prediction']].to_csv("submission.csv",index=False)

```
Data: EMP0
Features: similar aggregation like https://www.kaggle.com/code/ambrosm/amex-lightgbm-quickstart, except Cat aggs
Model: 5 fold, lgbm standard
Ensemble: choose max prediction value;highest proba out of 5 models
```